In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas_ml_quant
import pandas_quant_data_provider as qd

You should consider upgrading via the '/home/kic/sources/private/projects/pandas-quant/.venv/bin/python -m pip install --upgrade pip' command.
automatically imported pandas_quant_data_provider 0.3.0
automatically imported pandas_ta_quant_plot 0.3.0


('Requirement already satisfied: yfinance in /home/kic/sources/private/projects/pandas-quant/.venv/lib/python3.8/site-packages (0.1.64)\nRequirement already satisfied: lxml>=4.5.1 in /home/kic/.local/lib/python3.8/site-packages (from yfinance) (4.5.1)\nRequirement already satisfied: numpy>=1.15 in /home/kic/sources/private/projects/pandas-quant/.venv/lib/python3.8/site-packages (from yfinance) (1.20.3)\nRequirement already satisfied: pandas>=0.24 in /home/kic/sources/private/projects/pandas-quant/.venv/lib/python3.8/site-packages (from yfinance) (1.2.4)\nRequirement already satisfied: multitasking>=0.0.7 in /home/kic/.local/lib/python3.8/site-packages (from yfinance) (0.0.9)\nRequirement already satisfied: requests>=2.20 in /home/kic/.local/lib/python3.8/site-packages (from yfinance) (2.23.0)\nRequirement already satisfied: pytz>=2017.3 in /home/kic/.local/lib/python3.8/site-packages (from pandas>=0.24->yfinance) (2020.1)\nRequirement already satisfied: python-dateutil>=2.7.3 in /home/

In [2]:
print(pandas_ml_quant.__version__)
df = qd.fetch("SPY")[-500:]

0.3.0


In [3]:
with df[-500:].model() as m:
    from sklearn.neural_network import MLPClassifier

    from pandas_ml_common import random_splitter
    from pandas_ml_common.sampling.cross_validation import KEquallyWeightEvents
    from pandas_ml_utils import SkModelProvider, FittableModel, FeaturesLabels, ClassificationSummary, FittingParameter

    fit = m.fit(
        FittableModel(
            SkModelProvider(MLPClassifier(activation='tanh', hidden_layer_sizes=(60, 50), random_state=42, warm_start=True, max_iter=5)),
            FeaturesLabels(
                features=
                    [
                        lambda df: df["Close"].ta.macd().ML[['macd.*', 'signal.*']],
                        lambda df: df.ta.adx().ML[['+DI', '-DM', '+DM']],
                        lambda df: df["Close"].ta.mom(),
                        lambda df: df["Close"].ta.apo(),
                        lambda df: df.ta.atr(),
                        lambda df: df["Close"].ta.trix(),
                    ],
                features_postprocessor=lambda df: df.ta.rnn(280),
                labels=[
                    lambda df: df["Close"].ta.future_bband_quantile(include_mean=False).ta.one_hot_encode_discrete()
                ],
                reconstruction_targets=[
                    lambda df: df["Close"].ta.bbands()[["lower", "upper"]]
                ]
            ),
            summary_provider=ClassificationSummary.factory(False),
        ),
        FittingParameter(
            splitter=random_splitter(test_size=0.4, seed=42),
            cross_validation=KEquallyWeightEvents(n_splits=3),
            epochs=1
        )
    )

print("Done")

/home/kic/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kic/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/kic/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:582: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (5) reached and the optimization hasn't converged yet.
  warnings.warn(


Done


In [4]:
fit


Training Data,Test Data
,0
balanced_accuracy_score,0.28
cohen_kappa_score,-0.13
matthews_corrcoef,-0.14
f1_score,nan
jaccard_score,nan
precision_score,nan
recall_score,nan
accuracy_score,0.44
roc_auc_score,nan
